<a href="https://colab.research.google.com/github/rami-ramudu/tourist-chatbot/blob/main/HODO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import urllib
import bs4 as bs
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import warnings
warnings.filterwarnings("ignore")
import random
from sklearn.metrics.pairwise import cosine_similarity
from wikipedia import page
import random
import string 

import pandas as pd
import requests

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline
from pandas import DataFrame
import pyttsx3 
import speech_recognition as sr

from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=663570e76c67c7756e33837cbcbd2bd3683e432850f97c669f4240815723288e
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [ ]:
pip install pyttsx3

In [ ]:

!pip install SpeechRecognition #used for speech input
!pip install pyspellchecker #spelling checker

     |████████████████████████████████| 32.8 MB 38 kB/s 


# Text Gathering

## Getting weather details for the city

In [ ]:
page1=requests.get('https://www.timeanddate.com/weather/india')

In [ ]:
def temp(topic):
    
    page = page1
    soup = BeautifulSoup(page.content,'html.parser')

    data = soup.find(class_ = 'zebra fw tb-wt zebra va-m')

    tags = data('a')
    city = [tag.contents[0] for tag in tags]
    tags2 = data.find_all(class_ = 'rbi')
    temp = [tag.contents[0] for tag in tags2]

    indian_weather = pd.DataFrame(
    {
        'City':city,
        'Temperature':temp
    }
    )
    
    df = indian_weather[indian_weather['City'].str.contains(topic.title())] 
    
    return (df['Temperature'])

# Scrape City Details from Wikipedia

In [ ]:
def wiki_data(topic):
    
    topic=topic.title()
    topic=topic.replace(' ', '_',1)
    url1="https://en.wikipedia.org/wiki/"
    url=url1+topic

    source = urllib.request.urlopen(url).read()

    # Parsing the data/ creating BeautifulSoup object
    soup = bs.BeautifulSoup(source,'lxml')

    # Fetching the data
    text = ""
    for paragraph in soup.find_all('p'):
        text += paragraph.text

    import re
    # Preprocessing the data
    text = re.sub(r'\[[0-9]*\]',' ',text)
    text = re.sub(r'\s+',' ',text)
    text = text.lower()
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    
    
    return (text)

# Text Cleaning

## Removing special characters

In [ ]:
def rem_special(text):
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    return(text.translate(remove_punct_dict))

sample_text="I am sorry! I don't understand you."
rem_special(sample_text)

'I am sorry I dont understand you'

In [ ]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

In [ ]:
print(remove_punct_dict)

{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [ ]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

def stemmer(text):
    words = word_tokenize(text) 
    for w in words:
        text=text.replace(w,PorterStemmer().stem(w))
    return text

stemmer("He is Eating. He played yesterday. He will be going tomorrow.")

'He is eat. He play yesterday. He will be go tomorrow.'

In [ ]:
stemmer('I am playing a game.But I will be playing another game tomorrow')

'I am play a game.but I will be play anoth game tomorrow'

In [ ]:
lemmer = WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

sample_text="rocks corpora better" #default noun
LemTokens(nltk.word_tokenize(sample_text))

['rock', 'corpus', 'better']

In [ ]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()

stopword_list = nltk.corpus.stopwords.words('english')

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [ ]:
stopword_list = nltk.corpus.stopwords.words('english')

In [ ]:
stopword_list

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
tokens = tokenizer.tokenize("This is a sample sentence, showing off the stop words filtration.")
tokens = [token.strip() for token in tokens]

In [ ]:
print(tokens)

['This', 'is', 'a', 'sample', 'sentence', ',', 'showing', 'off', 'the', 'stop', 'words', 'filtration', '.']


In [ ]:
remove_stopwords("This is a sample sentence, showing off the stop words filtration.")

'sample sentence , showing stop words filtration .'

# Finding Parts of Speech

In [ ]:
import spacy 
spacy_df=[]
spacy_df1=[]
df_spacy_nltk=pd.DataFrame()
nlp = spacy.load('en_core_web_sm') 
  
# Process whole documents 
sample_text = ("The heavens are above. The moral code of conduct is above the civil code of conduct") 
doc = nlp(sample_text) 
  
# Token and Tag 
for token in doc:
    spacy_df.append(token.pos_)
    spacy_df1.append(token)


df_spacy_nltk['origional']=spacy_df1
df_spacy_nltk['spacy']=spacy_df
#df_spacy_nltk

# Name Entity Recognition

In [ ]:
import spacy 
nlp = spacy.load('en_core_web_sm') 

def ner(sentence):
    doc = nlp(sentence) 
    for ent in doc.ents: 
        print(ent.text, ent.label_) 
    

sentence = "A gangster family epic set in 1919 Birmingham, England; centered on a gang who sew razor blades in the peaks of their caps, and their fierce boss Tommy Shelby."
ner(sentence)

1919 DATE
Birmingham GPE
England GPE
Tommy Shelby PERSON


In [ ]:
doc = nlp("A gangster family epic set in 1919 Birmingham, England; centered on a gang who sew razor blades in the peaks of their caps, and their fierce boss Tommy Shelby.") 
for ent in doc.ents: 
    print(ent.text, ent.label_)
    print(ent.label)

1919 DATE
391
Birmingham GPE
384
England GPE
384
Tommy Shelby PERSON
380


# Sentiment Analysis Using Textblob

In [ ]:
from textblob import TextBlob

def senti(text):
    testimonial = TextBlob(text)
    return(testimonial.polarity)

sample_text="This apple is good"
print("polarity",senti(sample_text))
sample_text="This apple is not good"
print("polarity",senti(sample_text))

polarity 0.7
polarity -0.35


# Spelling Check

In [ ]:
from spellchecker import SpellChecker
spell = SpellChecker()


def spelling(text):
    splits = sample_text.split()
    for split in splits:
        text=text.replace(split,spell.correction(split))
        
    return (text)
    
    
sample_text="hapenning elephnt texte luckno sweeto"
spelling(sample_text)

'happenning elephant text lucknow sweet'

In [ ]:
!pip install pyspellchecker

# Tokenization

In [ ]:

#TOkenisation
print(nltk.sent_tokenize("Hey how are you? I am fine."))
print(nltk.word_tokenize("Hey how are you? I am fine."))

['Hey how are you?', 'I am fine.']
['Hey', 'how', 'are', 'you', '?', 'I', 'am', 'fine', '.']


# Word Embedding

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
documentA = 'This is about Messi'
documentB = 'This is about TFIDF'
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([documentA, documentB])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df

,about,is,messi,tfidf,this
0,0.448321,0.448321,0.630099,0.000000,0.448321
1,0.448321,0.448321,0.000000,0.630099,0.448321


# Conversation

## voice enabled

In [ ]:
def speak(message):
    engine= pyttsx3.init()
    engine.say('{}'.format(message))
    engine.runAndWait()

In [ ]:
engine = pyttsx3.init() 

engine.say("Hello hi") 
engine.runAndWait()

OSError: ignored